In [10]:
import pandas as pd
from sqlalchemy import create_engine
import yaml

In [11]:
# Read csv
trade_df = pd.read_csv("./data/trade.csv", encoding="utf-8", usecols=['BYRORGNM1', 'SPLYORGNM1'], skipinitialspace=True,
                       dtype={'HSCD': str, "QTY": float, 'BYRADDR2': str, 'SPLYADDR2': str})
trade_df.head()

,BYRORGNM1,SPLYORGNM1
0,주식회사 포스코대우,POSCO
1,주식회사 포스코대우,POSCO
2,주식회사 포스코대우,POSCO
3,주식회사 포스코대우,POSCO
4,주식회사 포스코대우,POSCO


In [12]:
csv_company_series = trade_df['BYRORGNM1'].append(trade_df['SPLYORGNM1'], ignore_index=True).drop_duplicates().reset_index(drop=True)
csv_company_series.head()

0         주식회사 포스코대우
1        도레이케미칼 주식회사
2    도레이첨단소재주식회사 4공장
3        도레이첨단소재주식회사
4    도레이첨단소재주식회사 2공장
dtype: object

In [14]:
len(csv_company_series)

53300

In [34]:
stopword_test = csv_company_series.str.replace("주식회사", "").str.replace("\(주\)", "")\
        .str.replace("\(유\)", "").str.replace("㈜", "").str.replace(" ", "")
stopword_test.head()

0         포스코대우
1        도레이케미칼
2    도레이첨단소재4공장
3       도레이첨단소재
4    도레이첨단소재2공장
dtype: object

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2,3))
tfidf_matrix = tfidf.fit_transform(stopword_test)

In [69]:
tfidf_matrix.shape

(53300, 92828)

In [70]:
tfidf.vocabulary_

{'주화': 71587,
 '강동오': 9281,
 '그오': 13182,
 '신마케': 47680,
 'h스': 4660,
 '디폴': 23594,
 '/한': 2095,
 '앤엠애': 52197,
 '알룩': 51270,
 '프린택': 86059,
 '계거래': 10503,
 '콜마피': 78276,
 '중경': 71739,
 '노스인': 16084,
 '보양엔': 34900,
 '오성': 57467,
 '솔모루': 43911,
 '이트무': 65579,
 '삼우강': 38869,
 'nr지': 6442,
 '인(l': 66016,
 '디팩토': 23582,
 '금호켐': 13718,
 'inu': 4878,
 '얄씨엔': 52622,
 '애랙트': 51817,
 '도볼': 20514,
 '팬월드': 84448,
 '서광테': 40004,
 '자장': 68190,
 '길안': 14237,
 '진씨': 73527,
 '천일페': 74978,
 '성문화': 41382,
 'veo': 8251,
 '팬벨트': 84439,
 '콘로라': 78190,
 '한피': 88979,
 '피아산': 86673,
 '박전자': 33171,
 '상중': 39554,
 '포케': 85183,
 '엔지어': 54741,
 '제로텍': 69631,
 '레건기': 25190,
 '터전기': 80804,
 '노브': 16043,
 '코이엔': 77966,
 '페셜티': 84681,
 '성통': 42131,
 '윈테': 61764,
 '창화학': 74703,
 '마상사': 29469,
 '스건기': 44967,
 '이더일': 63536,
 '서기계': 40017,
 '풍양상': 85698,
 'k국': 5514,
 '.ho': 1805,
 '벌로': 33965,
 '세비아': 42639,
 '도깨': 20435,
 '그린켐': 13141,
 '농가': 16318,
 '스택스': 46170,
 '림셰프': 28961,
 '예영농': 57113,
 '쏘시에': 49013,
 '서타': 404

In [67]:
from sklearn.metrics.pairwise import linear_kernel
company_i = 0
cosine_sim = linear_kernel(tfidf_matrix[company_i], tfidf_matrix)
print(csv_company_series[company_i])

주식회사 포스코대우


In [38]:
sorted_index = sorted(range(len(cosine_sim[0])), key=lambda i: cosine_sim[0][i])[-100:]
sorted_index.reverse()

In [46]:
for i in sorted_index:
    print(csv_company_series[i], str(cosine_sim[0][i]))

주식회사포스코대우 1.0
포스코 대우 1.0
(주)포스코대우 1.0
포스코대우 1.0
주식회사 포스코대우 1.0
주식회사 포스코대우T 0.8396449333639798
주식회사포스코 0.5883237370877368
(주)포스코 0.5883237370877368
주식회사 포스코 0.5883237370877368
주식회사 포스코대우 STS사업부 0.5675971756265995
포스코대우 TMC사업부 0.557354894125328
주식회사포스코대우STS 사업부 안산 0.5074446681797886
주식회사 포스코대우 STS 사업부 안산 0.5074446681797886
주식회사 포스코대우 STS사업부 안산 0.5074446681797886
(주)포스코대우 STS사업부 안산 0.5074446681797886
주식회사 포스코대우TMC 사업부 천안 0.4956238437481716
(주) 포스코대우 STS사업부 안산사업 0.45146490029844566
(주)포스코대우 STS 사업부 안산사업 0.45146490029844566
(주)포스코대우 STS사업부 부산사업장 0.4327141556997126
(주)포스코대우TMC사업부 포항사업장 0.42571366529612353
(주)포스코대우 TMC사업부 포항사업장 0.42571366529612353
㈜포스코대우 TMC사업부 포항사업장 0.42571366529612353
㈜포스코대우TMC사업부 천안사업장 0.425275886316196
주포스코 0.3670538604291159
(주)포스코건설 0.3159926135131049
포스코켐텍 0.3154629890037671
(주)포스코켐텍 0.3154629890037671
(주)포스코엠텍 0.31099774400245306
포스코강판주식회사 0.28720981865946255
포스코ICT 0.26951014573329385
(주)포스코플랜텍 0.26711273107732936
(주)포스코ESM 0.26328100572283303
주식회사포스코(포항) 0.227428587

In [33]:
for i in sorted_index:
    print(stopword_test[i])

포스코대우
포스코대우
포스코대우
포스코대우
포스코대우
포스코대우T
포스코대우STS
포스코대우TMC
포스코대우STS안산
포스코대우STS안산
포스코대우STS안산
포스코대우STS안산
포스코
포스코
포스코
포스코대우TMC천안
포스코대우STS안산사업
포스코대우STS안산사업
포스코대우STS부산사업장
포스코대우TMC포항사업장
포스코대우TMC포항사업장
포스코대우TMC포항사업장
포스코대우TMC천안사업장
주포스코
포스코건설
포스코켐텍
포스코켐텍
포스코엠텍
포스코강판
포스코ICT
포스코플랜텍
포스코ESM
포스코(포항)
포스코건설-본사
포스텍
포스텍
포스엔
포스프
포스포
포스포
포스켐
대우사
포스콤
포스테크
댄포스
댄포스
대우산업
대우산업
알포스
밴포스
대우상사
대우섬유
대우무역
대우정밀
대우공업
대우공업
포스메디
대우통상
케이포스
대우기전
대우건설
대우건설
텍스코
텍스코
텍스코
대우제약
대우제약
대우제약
대우제약
대우전장
포스프로
대우케미칼
대우엔텍
대우공업사
앤디포스
앤디포스
앤디포스
포스트레이드
포스트레이드
대우자원
타이포스
디어포스
디어포스
로이포스
엘포스텍
엠포스트
트리포스
대우ENG
대우ENG
대우볼트
포스디온
대우인터내셔널
대우인터내셔널
대우종광
포스링크
포스뱅크
포스톤텍
사인포스
포스텍전자
포스텍전자


In [75]:
test_name = ['포스코', '도레이케미칼', '포스코대우', '(주)우익반도체']
test_matrix = tfidf.transform(test_name)
test_matrix.shape

(4, 92828)

In [76]:
company_i = 0
cosine_sim = linear_kernel(test_matrix[company_i], tfidf_matrix)
print(csv_company_series[company_i])

주식회사 포스코대우


In [77]:
sorted_index = sorted(range(len(cosine_sim[0])), key=lambda i: cosine_sim[0][i])[-100:]
sorted_index.reverse()

In [78]:
for i in sorted_index:
    print(csv_company_series[i], str(cosine_sim[0][i]))

주식회사포스코 1.0
(주)포스코 1.0
주식회사 포스코 1.0
주포스코 0.6238977577992526
주식회사포스코대우 0.5883237370877369
포스코 대우 0.5883237370877369
(주)포스코대우 0.5883237370877369
포스코대우 0.5883237370877369
주식회사 포스코대우 0.5883237370877369
(주)포스코건설 0.537106687344116
포스코켐텍 0.536206461030013
(주)포스코켐텍 0.536206461030013
(주)포스코엠텍 0.5286166856736463
주식회사 포스코대우T 0.4939830450234805
포스코강판주식회사 0.4881832918746077
포스코ICT 0.4580983712596689
(주)포스코플랜텍 0.45402337903203593
(주)포스코ESM 0.44751042517186407
주식회사포스코(포항) 0.38657047692157703
(주)포스코건설-본사 0.36255959165907936
주식회사 포스코대우 STS사업부 0.33393089152508554
포스코대우 TMC사업부 0.3279051141959529
(주)포스텍 0.3157743004635862
주식회사 포스텍 0.3157743004635862
주식회사포스코대우STS 사업부 안산 0.2985417435487799
주식회사 포스코대우 STS 사업부 안산 0.2985417435487799
주식회사 포스코대우 STS사업부 안산 0.2985417435487799
(주)포스코대우 STS사업부 안산 0.2985417435487799
(주)포스엔 0.29363378318560013
주식회사 포스코대우TMC 사업부 천안 0.2915872719437129
(주)포스프 0.2911344418307872
주식회사 포스포 0.28796483725558736
(주)포스포 0.28796483725558736
(주)포스켐 0.28461818798943445
(주)포스콤 0.27444290306319064
(

In [44]:
pd.concat([csv_company_series, csv_company_series], axis=1, keys=['before', 'after'])

,before,after
0,주식회사 포스코대우,주식회사 포스코대우
1,도레이케미칼 주식회사,도레이케미칼 주식회사
2,도레이첨단소재주식회사 4공장,도레이첨단소재주식회사 4공장
3,도레이첨단소재주식회사,도레이첨단소재주식회사
4,도레이첨단소재주식회사 2공장,도레이첨단소재주식회사 2공장
5,도레이첨단소재주식회사 군산공장,도레이첨단소재주식회사 군산공장
6,티씨케이텍스타일 주식회사,티씨케이텍스타일 주식회사
7,티씨케이텍스타일(주),티씨케이텍스타일(주)
8,티씨케이텍스타일주식회사,티씨케이텍스타일주식회사
9,주식회사 에스엘라이팅,주식회사 에스엘라이팅


In [5]:
config = yaml.safe_load(open('dart_config.yml'))
db_url = 'mysql://' + config['DB_USER'] + ':' + config['DB_USER_PASSWORD'] + '@' + config['DB_HOST'] \
                            + '/' + config['DB_NAME'] + '?charset=utf8'
engine = create_engine(db_url, encoding='utf-8')
conn = engine.connect()

In [6]:
db_company_df = pd.read_sql_table(config['DB_TABLE_NAME_1'], con=conn, index_col='id', 
                          columns=['id', 'name', '업종', '상장일', '지역'])
db_company_df.head()

,name,업종,상장일,지역
id,,,,
1,롯데케미칼,기초 화학물질 제조업,1991-05-30,서울특별시
2,대한유화,기초 화학물질 제조업,1999-08-11,서울특별시
3,이화산업,기타 전문 도매업,1994-04-29,서울특별시
4,디케이앤디,플라스틱제품 제조업,2018-11-20,경기도
5,송원산업,기타 화학제품 제조업,1977-06-25,울산광역시


In [8]:
db_company_list = db_company_df['name'].drop_duplicates().reset_index(drop=True)
db_company_list.head()

0    롯데케미칼
1     대한유화
2     이화산업
3    디케이앤디
4     송원산업
Name: name, dtype: object

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3,3))
tfidf_matrix = tfidf.fit_transform(csv_company_list[:2000])

In [83]:
tfidf_matrix.shape

(2000, 6272)

In [84]:
tfidf.vocabulary_

{')신텍': 608,
 '탈링크': 5516,
 '제작소': 4814,
 '스지텍': 3183,
 '이엔텍': 4452,
 '은푸드': 4317,
 '기주식': 1532,
 '이앤엘': 4434,
 '타임리': 5509,
 '에스 ': 3718,
 '화이바': 6184,
 '티드 ': 5724,
 ' 기신': 36,
 'y t': 1303,
 '로라월': 2087,
 '지월드': 5131,
 'dis': 1051,
 'sco': 1240,
 ')탑스': 865,
 '주)크': 5002,
 'don': 1054,
 '주)혜': 5037,
 '아이디': 3501,
 '자동차': 4699,
 '진통상': 5207,
 '씨엠에': 3431,
 '피파트': 5964,
 '삼원티': 2797,
 'ngk': 1176,
 '벌테크': 2476,
 '군산공': 1472,
 '이노켐': 4334,
 ')피엔': 929,
 ' 대진': 47,
 '세코코': 2993,
 '주)브': 4913,
 '널(주': 1595,
 ')페타': 909,
 ' 토필': 231,
 '디에프': 1940,
 '유피피': 4305,
 '콘웍스': 5441,
 '엔아이': 3845,
 ' 하나': 251,
 '성인 ': 2936,
 '주)오': 4956,
 '세신어': 2972,
 'k상사': 1134,
 '원전선': 4213,
 ')진흥': 822,
 '일신화': 4661,
 ')아프': 646,
 '한백텍': 6055,
 '진현전': 5216,
 '정호섬': 4760,
 '산지사': 2761,
 '화패브': 6206,
 'ksm': 1132,
 '에이코': 3790,
 '미랜드': 2390,
 '호섬유': 6147,
 '프이브': 5911,
 '이글로': 4328,
 ')에프': 665,
 '상방산': 2824,
 ')가비': 337,
 '제이원': 4793,
 '솔코리': 3046,
 '스월드': 3168,
 '이치앤': 4521,
 '회사프': 6230,
 ')빠라': 529,
 '공신테':

In [85]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix[1], tfidf_matrix)

In [86]:
cosine_sim[0]

array([0.04649235, 1.        , 0.1422452 , ..., 0.        , 0.        ,
       0.        ])

In [87]:
sorted(range(len(cosine_sim[0])), key=lambda i: cosine_sim[0][i])[-5:]

[35, 3, 220, 18, 1]

In [99]:
csv_company_list[1]

'도레이케미칼 주식회사'

In [57]:
len(csv_company_list)

796